In [1]:
import numpy as np
import tensorflow as tf

In [2]:
train_vector=np.loadtxt('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro4\\PRNG-Dataset\\128-bit\\train_128bit.txt',delimiter=",")
validation_vector=np.loadtxt('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro4\\PRNG-Dataset\\128-bit\\val_128bit.txt',delimiter=",")
test_vector=np.loadtxt('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro4\\PRNG-Dataset\\128-bit\\test_128bit.txt',delimiter=",")

In [3]:
train_data=train_vector[:,:-1]
train_label=train_vector[:,-1]

validation_data=validation_vector[:,:-1]
validation_label=validation_vector[:,-1]

test_data=test_vector[:,:-1]
test_label=test_vector[:,-1]

vector_size=train_data.shape[1]

In [4]:
train_x=np.zeros((train_data.shape[0],vector_size,vector_size))
for i in range(train_data.shape[0]):
    train_x[i]=np.eye(vector_size)[np.array(train_data[i],dtype=np.int)]

test_x=np.zeros((test_data.shape[0],vector_size,vector_size))
for i in range(test_data.shape[0]):
    test_x[i]=np.eye(vector_size)[np.array(test_data[i],dtype=np.int)]

val_x=np.zeros((validation_data.shape[0],vector_size,vector_size))
for i in range(validation_data.shape[0]):
    val_x[i]=np.eye(vector_size)[np.array(validation_data[i],dtype=np.int)]

train_y=train_label-1
test_y=test_label-1
val_y=validation_label-1

In [5]:
x_placeholder=tf.placeholder(tf.float32,[None,vector_size,vector_size])
y_placeholder=tf.placeholder(tf.int32,[None])

basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=200)
outputs,states=tf.nn.dynamic_rnn(basic_cell,x_placeholder,dtype=tf.float32)

flatten=tf.layers.flatten(states)#none,100
#fc=tf.layers.dense(flatten,20,activation=tf.nn.relu)
logits=tf.layers.dense(flatten,8)
predicted_labels=tf.arg_max(logits,1)
losses=tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y_placeholder,8),logits=logits)
mean_loss=tf.reduce_mean(losses)
optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(losses)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use `tf.math.argmax` instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_feed_dict={x_placeholder:train_x,y_placeholder:train_y}
    val_feed_dict={x_placeholder:val_x,y_placeholder:val_y}
    max_acc=0
    for step in range(50):
        _,mean_loss_val,predicted_labels_val=sess.run([optimizer,mean_loss,predicted_labels],feed_dict=train_feed_dict)
        predicted_labels_validation=sess.run([predicted_labels],feed_dict=val_feed_dict)
        acc=np.sum(predicted_labels_validation==val_y)/val_y.shape[0]
        print(step,mean_loss_val,acc)
        if acc>max_acc:
            max_acc=acc
            print("saving model")
            saver.save(sess,'model_rnn/')

In [ ]:
with tf.Session() as sess:
    saver.restore(sess,'model_rnn/')
    test_feed_dict={x_placeholder:test_x,y_placeholder:test_y}
    predicted_labels_val=sess.run(predicted_labels,feed_dict=test_feed_dict)
    acc=np.sum(predicted_labels_val==test_y)/test_x.shape[0]
    print(acc)